<h1 align="center">Deep Learning - Master in Deep Learning of UPM</h1>

**IMPORTANTE**

Antes de empezar debemos instalar PyTorch Lightning, por defecto, esto valdría:

In [ ]:
!pip install pytorch-lightning

Además, si te encuentras ejecutando este código en Google Collab, lo mejor será que montes tu drive para tener acceso a los datos:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<h1 align="center">Atención - Introducción a los mecanismos de atención</h1>

En esta sesión práctica estudiaremos los principales mecanismos de atennción:
- Definición de 'Atención'
    - $Q$ query, $K$ key y $V$ value.
    - Alineación
- Mecanismos de atención
    - Atención aditiva
    - Dot product attention
    - Scaled dot product self-attention
$$ $$


## Carga del dataset
Se carga el dataset como en sesiones anteriores, no requiere hacer modificaciones. *Punto importante*: Las redes que incluyen mecanismos de atención pueden usarse también para secuencias.

In [ ]:
import datetime

import torch
import torch.nn as nn

import pytorch_lightning as pl
import torchmetrics
from pytorch_lightning import seed_everything

import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt


DATA_PATH = 'data/sales.csv'
SEED = 42
seed_everything(seed=SEED) # Fijamos una semilla para reproducibilidad en los experimentos

Vamos practicar con un dataset que contiene registros a lo largo de 10 años (cada día) del número de ventas en una tienda. A considerar:
- Estamos ante un conjunto de datos **secuencial** (depende de t) y **univariable**.
- La variable _sold_ será a la vez feature y etiqueta, siendo nuestra "X" los pasados w registros de _sold_ y la etiqueta el futuro.

In [ ]:
sales_df = pd.read_csv(DATA_PATH)
sales_df.head()

In [ ]:
sales_df['date'] = pd.to_datetime(sales_df['date'], format='%Y-%m-%d')
sales_df.sort_values('date', inplace=True)
print(f"Date range: {sales_df['date'].min()} to {sales_df['date'].max()}")

Para crear el dataset necesitamos definir tanto el tamaño de ventana como el de horizonte

In [ ]:
class SalesDataset(torch.utils.data.Dataset):
    def __init__(self, df, w=10, h=1):
        self.data = df.drop('date', axis=1).values
        self.w = w
        self.h = h

    def __len__(self):
        return len(self.data) - (self.w + self.h) + 1

    def __getitem__(self, idx):
        features = self.data[idx:idx+self.w] # [i: i+w)
        target = self.data[idx+self.w: idx+self.w+self.h] # [i+w, i+w+h)
        return features, target

Con ello ya podemos crear el DataModule de manera muy similar a como lo hacemos para datos tabulares.

El único cambio importante reside en como hacemos el particionado del dataset. No podemos hacer un particionado random ya que los datos necesitan estar en orden, por ello, los splits serán diferentes ventanas de tiempo ordenadas.

In [ ]:
class SalesDataModule(pl.LightningDataModule):
    def __init__(self, df, w=10, h=1, batch_size=16, val_size=0.2, test_size=0.2):
        super().__init__()
        self.train_df, self.val_df, self.test_df = self.sequential_train_val_test_split(df, val_size=val_size, test_size=test_size)
        self.scaler = self.normalize()

        self.w = w
        self.h = h

        self.batch_size = batch_size

    def setup(self, stage=None):
        if stage == 'fit':
            self.train_dataset = SalesDataset(self.train_df, w=self.w, h=self.h)
            self.val_dataset = SalesDataset(self.val_df, w=self.w, h=self.h)
        elif stage == 'test':
            self.test_dataset = SalesDataset(self.test_df, w=self.w, h=self.h)

    def sequential_train_val_test_split(self, df, val_size=0.2, test_size=0.2):
        # Aseguramos el formate de la fecha y ordenamos por ella
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
        df.sort_values('date', inplace=True)

        # Calculamos los ínidices para hacer los splits
        n = len(df)
        train_end = int((1 - val_size - test_size) * n)
        val_end = int((1 - test_size) * n)

        train = df.iloc[:train_end].copy()
        val = df.iloc[train_end:val_end].copy()
        test = df.iloc[val_end:].copy()

        return train, val, test

    def normalize(self):
        scaler = MinMaxScaler()
        self.train_df['sold'] = scaler.fit_transform(self.train_df['sold'].values.reshape(-1, 1))
        self.val_df['sold'] = scaler.transform(self.val_df['sold'].values.reshape(-1, 1))
        self.test_df['sold'] = scaler.transform(self.test_df['sold'].values.reshape(-1, 1))
        return scaler

    def collate_fn(self, batch):
        features, targets = zip(*batch)

        features = np.stack(features, axis=0)  # [batch_size, w, input_size]
        targets = np.stack(targets, axis=0)    # [batch_size, h, input_size]

        features = torch.tensor(features, dtype=torch.float32)
        targets = torch.tensor(targets, dtype=torch.float32)
        return features, targets

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, collate_fn=self.collate_fn)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn)

## Atención

La **atención** es un mecanismo que se puede incluir en redes neuronales que ayudará a la red a elegir qué partes de la entrada utilizar y seleccionar. Esta selección ocurre encontrando algún tipo de mecanismo que descubra cómo de alineados están la búsqueda (Query $Q$) y la clave (Key $K$). Queremos encontrar una función adecuada que nos diga cómo de importante es cada valor $V$.

El concepto de **alineamiento** viene dado por la función de alineamiento:
$score(Q, K)$ que indica la importancia de cada cruce entre busqueda y clave. Estamos observando qué valores recuperar, de la misma manera que haríamos en una base de datos.

### Algoritmo general
La atención aditiva será la primera en ser implementada. Computa el alineamiento entre la clave y el valor $(Q, V)$ respectivamente usando una capa lineal. Estas expresiones describen el alineamiento:

1. **Computar Alineamiento**:
    $$e_{t,i} = score(Q, K)$$

2. **Pesos de la atención**:
   $$
   \alpha_{t,i} = \frac{\exp(e_{t,i})}{\sum_{k=1}^{T} \exp(e_{t,k})}
   $$
   - $T$ es la longitud del tamaño de secuencia.

   Esta operación es equivalente a hacer una activación softmax `F.softmax(e_{t, i})`

3. **Computar la salida**:
   $$
   c_t,i = \alpha_{t,i} V_i
   $$
   donde $V_i$ es el valor de V en $i$ y $c_t$ es el vector de contexto (la salida) en el paso de tiempo $t$.

### Implementacion
Vamos a implementar un modulo de atención genérico:

In [ ]:
class Attention(nn.Module):
    """
    Modulo de atencion genérico
    hidden_dim[int]: tamaño de la representación
    """
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim

    def _score(self, q, k):
        # Q = Query, K = Key
        # Las dimensiones de Q y K tienen que ser compatibles!
        pass # Dependiendo de la funcion score tnedremos un mecanismo u otro

    def forward(self, q, k, v):
        # Q = Query, K = Key, V = Value
        # Las dimensiones de Q, K y V tienen que ser compatibles!
        score = self._score(q, k)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * v
        return context_vector, attention_weights

## Módulos de atención

### Aditiva
La atención aditiva será la primera en ser implementada. Computa el alineamiento entre la clave y el valor $(Q, V)$ respectivamente usando una capa lineal y una activación `tanh`. Estas expresiones describen el alineamiento:

$$score(Q, K) = v_a^T tanh(W_a[Q, K])$$

In [ ]:
class AdditiveAttention(nn.Module):
    """
    Modulo de atencion
    hidden_dim[int]: tamaño de la representación
    """
    def __init__(self, hidden_dim):
        super(AdditiveAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.W1 = nn.Linear(hidden_dim, hidden_dim)
        self.W2 = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
    def _score(self, q, k):
        # Q = Query, K = Key
        # Las dimensiones de Q y K tienen que ser compatibles!
        # Mecanismo de la atencion aditiva
        return self.V(torch.tanh(self.W1(q) + self.W2(k)))

    def forward(self, q, k, v):
        # Q = Query, K = Key, V = Value
        # Las dimensiones de Q, K y V tienen que ser compatibles!
        score = self._score(q, k) # Q[batch_size; seq_len, hidden_dim]
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * v # Una atencion para cada elem. de la secuencia
        return context_vector, attention_weights #C[batch_size; seq_len; hidden_dim] // #A[batch_size; seq_len]

In [ ]:
with torch.no_grad():
  dummy = AdditiveAttention(16)
  X = torch.rand(1, 8, 16)
  print(dummy(X, X, X))

### General
La atención general es la siguiente en ser implementada. Computa el alineamiento como dos productos de matrices siguiendo la expresión:
$$score(Q, K) = Q^TW_aK $$

In [ ]:
class GeneralAttention(nn.Module):
    def __init__(self, query_dim, key_dim):
        super(GeneralAttention, self).__init__()
        self.W_a = nn.Parameter(torch.randn(query_dim, key_dim))

    def _score(self, q, k):
        # Q = Query, K = Key
        # Las dimensiones de Q y K tienen que ser compatibles!
        # Mecanismo de la atencion general
        left = torch.matmul(q, self.W_a)
        return torch.matmul(left, k.transpose(-2, -1))

    def forward(self, q, k, v):
        # Q = Query, K = Key, V = Value
        # Las dimensiones de Q, K y V tienen que ser compatibles!
        score = self._score(q, k) #C[batch_size; seq_len; hidden_dim]
        attention_weights = torch.softmax(score, dim=1)
        # Cuidado, esto es el producto matricial!
        context_vector = torch.matmul(attention_weights, v)
        return context_vector, attention_weights #C[batch_size; seq_len; hidden_dim] // #A[batch_size; seq_len; seq_len]

In [ ]:
with torch.no_grad():
  dummy = GeneralAttention(16,16)
  X = torch.rand(1, 8, 16)
  print(dummy(X, X, X))

### Scaled Dot Product
Este mecanismo es frecuentemente utilizado en redes modernas (transformers).Computa el alineamiento como el producto matricial entre clave y búsqueda, después escala mediante la dimensión:
$$score(Q, K) = \frac{QK^T}{\sqrt{d_K}} $$
En este caso $d_K$ es la dimensión de la clave.

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def _score(self, q, k):
        # Q = Query, K = Key
        # Las dimensiones de Q y K tienen que ser compatibles!
        # Mecanismo de la atencion general
        return torch.matmul(q, k.transpose(-2, -1)) / np.sqrt(q.size(-1))

    def forward(self, q, k, v):
        # Q = Query, K = Key, V = Value
        # Las dimensiones de Q, K y V tienen que ser compatibles!
        score = self._score(q, k) #C[batch_size; seq_len; hidden_dim]
        attention_weights = torch.softmax(score, dim=1)
        # Cuidado, esto es el producto matricial!
        context_vector = torch.matmul(attention_weights, v)
        return context_vector, attention_weights #C[batch_size; seq_len; hidden_dim] // #A[batch_size; seq_len; seq_len]

In [ ]:
with torch.no_grad():
  dummy = ScaledDotProductAttention()
  X = torch.rand(1, 8, 16)
  print(dummy(X, X, X))

## LSTM con atencion
Vamos a darle a una LSTM la capacidad de tener (auto-)atención en su última salida. Inyectamos uno de los módulos de atención a la Lstm. Un ejemplo con el SDPA (Scaled dot-product attention):

In [ ]:
class LSTMAttentionRegressor(nn.Module):
    """
    LSTM Regressor model
    h[int]: horizonte de predicción
    input_size[int]: variables de la serie temporal
    hidden_size[int]: tamaño de las capas ocultas de la RNN
    num_layers[int]: número de capas de la RNN (si > 1, stacking de células RNN)
    batch_first[bool]: si el batch_size es la primera dimensión
    p_drop[float]: probabilidad de dropout
    """
    def __init__(self,  h=1,
                 input_size=1,
                 hidden_size=64,
                 num_layers=1,
                 batch_first=True,
                 p_drop=0.0,
                 enable_attention=True):
        super(LSTMAttentionRegressor, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=batch_first,
                            dropout=p_drop,
                            )
        self.sdpa = ScaledDotProductAttention()
        self.fc = nn.Linear(hidden_size, h)
        self.enable_attention = enable_attention

    def forward(self, x):
        x, _ = self.lstm(x)
        if self.enable_attention:
          x, _ = self.sdpa(x, x, x)
        output = self.fc(x[:, -1, :]) # Elegir la última salida
        return output #out[batch_size; h]

In [ ]:
with torch.no_grad():
  dummy = LSTMAttentionRegressor(h=2, input_size=16, hidden_size=16, num_layers=1)
  X = torch.rand(4, 8, 16)
  print(dummy(X).shape)

Vamos a declarar dos redes idénticas, una de ellas con un mecanismo de atención, y otra sin dicho mecanismo.
Probad a cambiar el tamaño de ventana comparando como crece el número de parámetros necesarios para modelar los datos!

In [ ]:
input_size = 3
hidden_size = 64
batch_first = True
num_layers = 1
batch_size = 64
w = 10
h = 2

pooling = 'last'

lstm_no_att = LSTMAttentionRegressor(h=h,
                                     input_size=input_size,
                                     hidden_size=hidden_size,
                                     num_layers=num_layers,
                                     batch_first=batch_first,
                                     )
lstm_w_att = LSTMAttentionRegressor(h=h,
                                     input_size=input_size,
                                     hidden_size=hidden_size,
                                     num_layers=num_layers,
                                     batch_first=batch_first,
                                     enable_attention=True,
                                    )
x = torch.randn(batch_size, w, input_size)

def stats(model, x):
    name = model.__class__.__name__
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    print(f'Model: {name}')
    print(f'Model Parameters: {params}')
    print(f'Model Output Size: {model(x).size()}', end='\n\n')

stats(lstm_no_att, x)
stats(lstm_w_att, x) # Esta vez nuestros modelos son identicos, salvo la atencion!

## **EXTRA**: LSTM (con atencion entrenable)
Se puede entrenar parte de la atencion para mejorar el entendimiento usando matrices de pesos.

Usala con el codigo anterior

In [ ]:
class LSTMTrainableAttentionRegressor(nn.Module):
    """
    LSTM Regressor model
    h[int]: horizonte de predicción
    input_size[int]: variables de la serie temporal
    hidden_size[int]: tamaño de las capas ocultas de la RNN
    num_layers[int]: número de capas de la RNN (si > 1, stacking de células RNN)
    batch_first[bool]: si el batch_size es la primera dimensión
    p_drop[float]: probabilidad de dropout
    """
    def __init__(self,  h=1,
                 input_size=1,
                 hidden_size=64,
                 num_layers=1,
                 batch_first=True,
                 p_drop=0.0,
                 enable_attention=True):
        super(LSTMTrainableAttentionRegressor, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=batch_first,
                            dropout=p_drop,
                            )
        self.sdpa = ScaledDotProductAttention()
        self.Qw = nn.Linear(hidden_size, hidden_size)
        self.Kw = nn.Linear(hidden_size, hidden_size)
        self.Vw = nn.Linear(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, h)
        self.enable_attention = enable_attention

    def forward(self, x):
        x, _ = self.lstm(x)
        if self.enable_attention:
          x, _ = self.sdpa(self.Qw(x), self.Kw(x), self.Vw(x))
        output = self.fc(x[:, -1, :]) # Elegir la última salida
        return output #out[batch_size; h]

## Entrenamiento
Vamos a hacer el entrenamiento, definiendo el modulo de lighting y el resto de detalles faltantes para completar el proceso.

In [ ]:
class SalesPredictor(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters() # guardamos la configuración de hiperparámetros
        self.learning_rate = learning_rate
        self.model = model
        self.criterion = nn.MSELoss()
        self.r2 = torchmetrics.R2Score()

    def forward(self, x):
        return self.model(x)

    def compute_batch(self, batch, split='train'):
        inputs, targets = batch
        output = self(inputs)

        preds = output.view(-1)
        targets = targets.view(-1)

        loss = self.criterion(preds, targets)
        self.log_dict(
            {
                f'{split}_loss': loss,
                f'{split}_r2': self.r2(preds, targets),
            },
            on_epoch=True, prog_bar=True)

        return loss

    def training_step(self, batch, batch_idx):
        return self.compute_batch(batch, 'train')

    def validation_step(self, batch, batch_idx):
        return self.compute_batch(batch, 'val')

    def test_step(self, batch, batch_idx):
        return self.compute_batch(batch, 'test')

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate) # self.parameters() son los parámetros del modelo

### Bucle de entrenamiento

In [ ]:
# Parámetros
SAVE_DIR = f'lightning_logs/sales/{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}'
w = 50
h = 3
batch_size = 64
num_layers = 1
hidden_size = 128
learning_rate = 1e-3
p_drop = 0.2

# DataModule
data = pd.read_csv(DATA_PATH)
data_module = SalesDataModule(data, w=w, h=h, batch_size=batch_size)

# Model (Probad a descomentar uno u otro)
#lstm = LSTMTrainableAttentionRegressor(...) # Usa esta otra definicion mas adelante
lstm = LSTMTrainableAttentionRegressor(h=h,
                              input_size=1,
                              hidden_size=hidden_size,
                              num_layers=num_layers,
                              batch_first=True,
                              enable_attention=True, # Cambia a false para probar que ocurre!
                              )
# model = MLPRegressor(w=w, h=h, input_size=1, hidden_size=hidden_size)

# LightningModule
module = SalesPredictor(lstm, learning_rate=learning_rate)

# Callbacks
early_stopping_callback = pl.callbacks.EarlyStopping(
    monitor='val_loss', # monitorizamos la pérdida en el conjunto de validación
    mode='min',
    patience=5, # número de epochs sin mejora antes de parar
    verbose=False, # si queremos que muestre mensajes del estado del early stopping
)
model_checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_loss', # monitorizamos la pérdida en el conjunto de validación
    mode='min', # queremos minimizar la pérdida
    save_top_k=1, # guardamos solo el mejor modelo
    dirpath=SAVE_DIR, # directorio donde se guardan los modelos
    filename=f'best_model' # nombre del archivo
)

callbacks = [early_stopping_callback, model_checkpoint_callback]

# Loggers
csv_logger = pl.loggers.CSVLogger(
    save_dir=SAVE_DIR,
    name='metrics',
    version=None
)

loggers = [csv_logger] # se pueden poner varios loggers (mirar documentación)

# Trainer
trainer = pl.Trainer(max_epochs=50, accelerator='cpu', callbacks=callbacks, logger=loggers)

trainer.fit(module, data_module)
results = trainer.test(module, data_module)

### Visualizacion
Visualicemos con Matplotlib como están ajustando las predicciones del modelo a la realidad. Cuidado, el modelo de atencion no tiene por que ser mejor que el modelo sin atención. La atención es principalmente beneficiosa en problemas más complejos!

In [ ]:
all_preds = []
all_targets = []

# Iteramos sobre el test_dataloader
with torch.no_grad():
    for batch in data_module.test_dataloader():
        features, targets = batch
        # features: [batch_size, seq_len, input_size]
        # targets: [batch_size, horizon] (o [batch_size] si es un horizonte 1)
        features = features.to(module.device)
        targets = targets.to(module.device)

        preds = module(features)  # [batch_size, horizon] o [batch_size]

        # Guardamos predicciones y targets
        all_preds.append(preds.cpu())
        all_targets.append(targets.cpu())

# Concatenamos todos los batch para tener un solo tensor
# Si es horizonte > 1, puedes elegir un paso en particular o hacer promedio
# Ej: tomamos el primer paso:
all_preds = torch.cat(all_preds, dim=0)[:, 0]
all_targets = torch.cat(all_targets, dim=0)[:, 0]

# Utilizamos el scaler inverso para obtener los valores originales
# all_preds = dm.scaler_test.inverse_transform(all_preds.reshape(-1, 1)).flatten()
# all_targets = dm.scaler_test.inverse_transform(all_targets.reshape(-1, 1)).flatten()

# Ahora graficamos
plt.figure(figsize=(10, 6))
plt.plot(all_targets, label='Real')
plt.plot(all_preds, label='Predicción')
plt.title('Comparación de Predicción vs Real (Test)')
plt.xlabel('Índice de muestra')
plt.ylabel('Valor')
plt.legend()
plt.show()

In [ ]:
all_preds = []
all_targets = []

# Iteramos sobre el test_dataloader
with torch.no_grad():
    for batch in data_module.test_dataloader():
        features, targets = batch
        # features: [batch_size, seq_len, input_size]
        # targets: [batch_size, horizon] (o [batch_size] si es un horizonte 1)
        features = features.to(module.device)
        targets = targets.to(module.device)

        preds = module(features)  # [batch_size, horizon] o [batch_size]

        # Guardamos predicciones y targets
        all_preds.append(preds.cpu())
        all_targets.append(targets.cpu())

# Concatenamos todos los batch para tener un solo tensor
# Si es horizonte > 1, puedes elegir un paso en particular o hacer promedio
# Ej: tomamos el primer paso:
all_preds = torch.cat(all_preds, dim=0)[:, 0]
all_targets = torch.cat(all_targets, dim=0)[:, 0]

# Utilizamos el scaler inverso para obtener los valores originales
# all_preds = dm.scaler_test.inverse_transform(all_preds.reshape(-1, 1)).flatten()
# all_targets = dm.scaler_test.inverse_transform(all_targets.reshape(-1, 1)).flatten()

# Ahora graficamos
plt.figure(figsize=(10, 6))
plt.plot(all_targets, label='Real')
plt.plot(all_preds, label='Predicción')
plt.title('Comparación de Predicción vs Real (Test)')
plt.xlabel('Índice de muestra')
plt.ylabel('Valor')
plt.legend()
plt.show()